In [0]:
external_azure_path = 'abfss://container20260122@datalake20260121.dfs.core.windows.net/'

dbutils.fs.ls(f'{external_azure_path}/02-silver')

In [0]:
# Creating silver tmp tables to create gold analytic data
for delta_table in dbutils.fs.ls(f'{external_azure_path}/02-silver'):
    (spark.read
        .format("delta")
        .load(delta_table.path)
        .createOrReplaceTempView(f'tmp_silver_{delta_table.name.rstrip('/')}')
    )

In [0]:
%py
df_gold_pending_orders = spark.sql('''
    -- Creating SQL query to aggregate order data                            
    WITH cte_pending_orders AS (
        SELECT 
              customer_id
            , order_date
            , SUM(item_quantity) AS item_quantity
            , store_name
        FROM 
            tmp_silver_orders
        WHERE
            status = 'Pending'
        GROUP BY
              customer_id
            , store_name
            , order_date
    )
    SELECT 
          po.*
        , ct.first_name AS customer_first_name
        , ct.email
        , ct.phone
    FROM
        cte_pending_orders              po
        LEFT JOIN tmp_silver_customers  ct ON po.customer_id = ct.customer_id 
    WHERE  
        ct.email        IS NOT NULL
        AND ct.phone    IS NOT NULL                    
''')

display(df_gold_pending_orders)

In [0]:
# Writing on silver azure external storage in delta format
(df_gold_pending_orders.write
    .mode('overwrite')
    .format('delta')
    .option('mergeSchema', 'true')
    .save(f'{external_azure_path}/03-gold/pending-orders')
)

In [0]:
%sql
DROP TABLE IF EXISTS bike_store.gold_schema.pending_orders

In [0]:
%sql
-- Creating databricks delta table
CREATE TABLE IF NOT EXISTS bike_store.gold_schema.pending_orders
LOCATION 'abfss://container20260122@datalake20260121.dfs.core.windows.net/03-gold/pending-orders'